# 如何在聊天模型中使用少量示例

本指南介绍了如何使用示例输入和输出来提示聊天模型。为模型提供几个这样的示例称为少量示例提示（few-shotting），是一种简单但强大的引导生成方法，有时可以显著提升模型性能。

目前对于如何最好地进行少量示例提示似乎还没有明确的共识，最佳的提示编译方式可能因模型而异。因此，我们提供了像 [FewShotChatMessagePromptTemplate](https://api.js.langchain.com/classes/langchain_core.prompts.FewShotChatMessagePromptTemplate.html) 这样的少量示例提示模板，作为灵活的起点，您可以根据需要进行修改或替换。

少量示例提示模板的目标是根据输入动态选择示例，然后将这些示例格式化为最终提示提供给模型。

**注意：** 以下代码示例仅适用于聊天模型，因为 `FewShotChatMessagePromptTemplates` 被设计为输出格式化的 [聊天消息](/docs/concepts/messages)，而不仅仅是纯字符串。有关适用于补全模型（LLMs）的纯字符串模板的类似少量示例提示示例，请参阅 [少量示例提示模板](/docs/how_to/few_shot_examples/) 指南。

:::info 预备知识

本指南假设您已熟悉以下概念：

- [提示模板](/docs/concepts/prompt_templates)
- [示例选择器](/docs/concepts/example_selectors)
- [聊天模型](/docs/concepts/chat_models)
- [向量存储](/docs/concepts/#vectorstores)

:::

## 固定示例

最基本的（也是最常见的）少量示例提示技术是使用固定的提示示例。这种方式可以让你选择一个链（chain），对其进行评估，并避免在生产环境中担心其他额外的变动因素。

模板的基本组成部分包括：
- `examples`：需要包含在最终提示中的示例对象数组。
- `examplePrompt`：通过其 [`formatMessages`](https://api.js.langchain.com/classes/langchain_core.prompts.FewShotChatMessagePromptTemplate.html#formatMessages) 方法将每个示例转换为一条或多条消息。常见的示例是将每个示例转换为一条人类消息和一条AI消息回复，或者一条人类消息后跟一条函数调用消息。

以下是一个简单的演示。首先，定义你想要包含的示例：

In [4]:
import {
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
} from "@langchain/core/prompts"

const examples = [
    { input: "2+2", output: "4" },
    { input: "2+3", output: "5" },
]

接下来，将它们组装成少样本提示模板。

In [7]:
// This is a prompt template used to format each individual example.
const examplePrompt = ChatPromptTemplate.fromMessages(
    [
        ["human", "{input}"],
        ["ai", "{output}"],
    ]
)
const fewShotPrompt = new FewShotChatMessagePromptTemplate({
    examplePrompt,
    examples,
    inputVariables: [], // no input variables
})

const result = await fewShotPrompt.invoke({});
console.log(result.toChatMessages())

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: { content: "2+2", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "2+2",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "4",
      tool_calls: [],
      invalid_tool_calls: [],
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "4",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {},
    tool_calls: [],
    invalid_tool_calls: []
  },
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: { content: "2+3", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "2+3",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs:

最后，我们按如下方式组装最终提示，将 `fewShotPrompt` 直接传递给 `fromMessages` 工厂方法，并将其与模型一起使用：

In [8]:
const finalPrompt = ChatPromptTemplate.fromMessages(
    [
        ["system", "You are a wondrous wizard of math."],
        fewShotPrompt,
        ["human", "{input}"],
    ]
)

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs />
```

In [9]:
const chain = finalPrompt.pipe(model);

await chain.invoke({ input: "What's the square of a triangle?" })

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "A triangle does not have a square. The square of a number is the result of multiplying the number by"... 8 more characters,
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "A triangle does not have a square. The square of a number is the result of multiplying the number by"... 8 more characters,
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 23, promptTokens: 52, totalTokens: 75 },
    finish_reason: "stop"
  },
  tool_calls: [],
  invalid_tool_calls: []
}

## 动态少样本提示

有时你可能希望根据输入内容从整体示例集中仅选择几个示例进行展示。在这种情况下，可以将传递给 `FewShotChatMessagePromptTemplate` 的 `examples` 替换为 `exampleSelector`。其他组件则与上述保持一致！我们动态的少样本提示模板如下所示：

- `exampleSelector`：负责根据给定输入选择少样本示例（以及它们的返回顺序）。这些组件实现了 [BaseExampleSelector](https://api.js.langchain.com/classes/langchain_core.example_selectors.BaseExampleSelector.html) 接口。一个常见的例子是基于向量存储的 [SemanticSimilarityExampleSelector](https://api.js.langchain.com/classes/langchain_core.example_selectors.SemanticSimilarityExampleSelector.html)
- `examplePrompt`：通过其 [`formatMessages`](https://api.js.langchain.com/classes/langchain_core.prompts.FewShotChatMessagePromptTemplate.html#formatMessages) 方法将每个示例转换为一个或多个消息。一个常见的例子是将每个示例转换为一个人类消息和一个AI消息响应，或者是一个人类消息后跟一个函数调用消息。

这些组件同样可以与其他消息和聊天模板组合，以构建最终的提示。

让我们通过一个 `SemanticSimilarityExampleSelector` 的示例来演示。由于此实现使用向量存储根据语义相似性选择示例，因此我们首先需要填充该存储。基本思路是我们希望搜索并返回与文本输入最相似的示例，因此我们会对提示示例的 `values` 进行嵌入，而不是考虑其键：

In [19]:
import { SemanticSimilarityExampleSelector } from "@langchain/core/example_selectors";
import { MemoryVectorStore } from "langchain/vectorstores/memory";
import { OpenAIEmbeddings } from '@langchain/openai';

const examples = [
  { input: '2+2', output: '4' },
  { input: '2+3', output: '5' },
  { input: '2+4', output: '6' },
  { input: 'What did the cow say to the moon?', output: 'nothing at all' },
  {
    input: 'Write me a poem about the moon',
    output: 'One for the moon, and one for me, who are we to talk about the moon?',
  },
];

const toVectorize = examples.map((example) => `${example.input} ${example.output}`);
const embeddings = new OpenAIEmbeddings();
const vectorStore = await MemoryVectorStore.fromTexts(toVectorize, examples, embeddings);

### 创建 `exampleSelector`

有了创建好的向量存储后，我们就可以创建 `exampleSelector`。在这里，我们将单独调用它，并设置其 `k` 参数，以仅获取与输入最接近的两个示例。

In [21]:
const exampleSelector = new SemanticSimilarityExampleSelector(
    {
        vectorStore,
        k: 2
    }
)

// The prompt template will load examples by passing the input do the `select_examples` method
await exampleSelector.selectExamples({ input: "horse"})

[
  {
    input: "What did the cow say to the moon?",
    output: "nothing at all"
  },
  { input: "2+4", output: "6" }
]

### 创建提示模板

我们现在使用上面创建的 `exampleSelector` 来组装提示模板。

In [23]:
import {
    ChatPromptTemplate,
    FewShotChatMessagePromptTemplate,
} from "@langchain/core/prompts"

// Define the few-shot prompt.
const fewShotPrompt = new FewShotChatMessagePromptTemplate({
    // The input variables select the values to pass to the example_selector
    inputVariables: ["input"],
    exampleSelector,
    // Define how ech example will be formatted.
    // In this case, each example will become 2 messages:
    // 1 human, and 1 AI
    examplePrompt: ChatPromptTemplate.fromMessages(
        [["human", "{input}"], ["ai", "{output}"]]
    ),
})

const results = await fewShotPrompt.invoke({ input: "What's 3+3?" });
const fewShotMessages = results.toChatMessages()
console.log(fewShotMessages)

[
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: { content: "2+3", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "2+3",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "5",
      tool_calls: [],
      invalid_tool_calls: [],
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "5",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {},
    tool_calls: [],
    invalid_tool_calls: []
  },
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: { content: "2+2", additional_kwargs: {}, response_metadata: {} },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "2+2",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs:

我们可以将这个少量示例的聊天消息提示模板传递给另一个聊天提示模板：

In [24]:
const finalPrompt = ChatPromptTemplate.fromMessages(
    [
        ["system", "You are a wondrous wizard of math."],
        ...fewShotMessages,
        ["human", "{input}"],
    ]
)

const result = await finalPrompt.invoke({ input: "What's 3+3?" });
console.log(result)

ChatPromptValue {
  lc_serializable: true,
  lc_kwargs: {
    messages: [
      HumanMessage {
        lc_serializable: true,
        lc_kwargs: {
          content: "2+3",
          additional_kwargs: {},
          response_metadata: {}
        },
        lc_namespace: [ "langchain_core", "messages" ],
        content: "2+3",
        name: undefined,
        additional_kwargs: {},
        response_metadata: {}
      },
      AIMessage {
        lc_serializable: true,
        lc_kwargs: {
          content: "5",
          tool_calls: [],
          invalid_tool_calls: [],
          additional_kwargs: {},
          response_metadata: {}
        },
        lc_namespace: [ "langchain_core", "messages" ],
        content: "5",
        name: undefined,
        additional_kwargs: {},
        response_metadata: {},
        tool_calls: [],
        invalid_tool_calls: []
      },
      HumanMessage {
        lc_serializable: true,
        lc_kwargs: {
          content: "2+2",
          addition

### 与聊天模型一起使用

最后，你可以将你的模型连接到少量样本提示。

```{=mdx}
<ChatModelTabs
  customVarName="模型"
/>
```

In [25]:
const chain = finalPrompt.pipe(model);

await chain.invoke({ input: "What's 3+3?" })

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: "6",
    tool_calls: [],
    invalid_tool_calls: [],
    additional_kwargs: { function_call: undefined, tool_calls: undefined },
    response_metadata: {}
  },
  lc_namespace: [ "langchain_core", "messages" ],
  content: "6",
  name: undefined,
  additional_kwargs: { function_call: undefined, tool_calls: undefined },
  response_metadata: {
    tokenUsage: { completionTokens: 1, promptTokens: 51, totalTokens: 52 },
    finish_reason: "stop"
  },
  tool_calls: [],
  invalid_tool_calls: []
}

## 下一步
你现在已经学会了如何在聊天提示中添加少量示例。

接下来，请查看本节中有关提示模板的其他操作指南、关于[使用文本生成模型进行少量示例](/docs/how_to/few_shot_examples)的相关操作指南，或[示例选择器操作指南](/docs/how_to/example_selectors/)的其他内容。